# Goals in cleaning up the data.
1. Extract GPS coordinates from Location column (street address)
2. Consolidate number of Incidents (group similar e.g Theft from Motor Vehicle vs. Theft)


In [1]:
import pandas as pd
import numpy as np
import re
from pprint import pprint

In [2]:
# what am I using?

import pkg_resources
modules_used = 'pandas,numpy,geopy,folium'.split(',')
print('Versions')
for mod in modules_used:
    print('{}:\t{}'.format(mod, pkg_resources.get_distribution(mod).version))

Versions
pandas:	0.23.4
numpy:	1.15.2
geopy:	1.17.0
folium:	0.6.0


In [3]:
# import data
df = pd.read_json('crimeLogData.json')
df.shape

(11240, 7)

In [4]:
# see columns
df.columns

Index(['Comments', 'Disposition', 'Incident', 'Location', 'Occured',
       'Reported', 'UCPD_ID'],
      dtype='object')

In [5]:
# (ignore the fact that Occurred is misspelled smh)
df.rename(columns={'Occured':'Occurred'}, inplace=True)
df.columns

Index(['Comments', 'Disposition', 'Incident', 'Location', 'Occurred',
       'Reported', 'UCPD_ID'],
      dtype='object')

In [6]:
# check data types per column
df.dtypes

# BAD: Occured and Reported are not date-time type

Comments       object
Disposition    object
Incident       object
Location       object
Occurred       object
Reported       object
UCPD_ID        object
dtype: object

In [7]:
# peek inside
df.head()

,Comments,Disposition,Incident,Location,Occurred,Reported,UCPD_ID
0,Woman reports losing a Canon Power Shot digita...,Closed,Lost Property,5810 S. University (Quad),6/28/10 2:45 PM,7/1/10 12:42 PM,W0731
1,Unknown person smashed windshield of a rental ...,Open,Criminal Damage to Vehicle,61st & Drexel,6/29/10 to 7/1/10 3:00 PM to 1:50 PM,7/1/10 2:01 PM,W0732
2,Unknown person smashed rear passenger's side w...,Open,Theft from Motor Vehicle,60th between Ingleside & Ellis,6/29/10 to 7/1/10 5:30 PM to 12:30 PM,7/1/10 2:48 PM,W0733
3,Bicycle taken from bike rack,Open,Theft,60th and Ellis,6/23/10 to 6/29/10 9:00 AM to 5:00 PM,7/1/10 3:13 PM,W0734
4,"Bottle slipped from man's hands, broke causing...",Closed,Chemical Spill,924 E. 57th St. (BSLC),7/1/10 6:15 PM,7/1/10 6:15 PM,W0735


In [8]:
# remove bad data
'''
NaN corresponds to "No Incident Reports" & missing data, and
voids to some internal error
'''
df[9010:9020]  # preview

,Comments,Disposition,Incident,Location,Occurred,Reported,UCPD_ID
9010,"Individual reports losing iPhone, possibly on ...",Closed,Lost Property,5640 S. Ellis to 56th & Cornell,3/17/17 10:00 PM to 11:00 PM,3/18/17 10:55 AM,17-00215
9011,Individual collapsed during a track meet / Tra...,Closed,Medical Call,5550 S. University (Crown Field House),3/18/17 12:45 PM,3/18/17 12:45 PM,17-00216
9012,3 unknown suspects struck victim with fists an...,CPD,Information / Robbery / Vehicle Hijacking,5558 S. Kimbark,3/18/17 4:05 PM,3/18/17 4:07 PM,17-00217
9013,Number generated in error,Void,Void,Void,Void,Void,17-00218
9014,NaN,NaN,:,NaN,NaN,NaN,NaN
9015,NaN,NaN,No Incident Reports this date 3/19/17,NaN,NaN,NaN,NaN
9016,NaN,NaN,:,NaN,NaN,NaN,NaN
9017,11 aluminum support beams taken from bleachers...,Open,Theft,821 E. 55th St. (Stagg Field),3/17/17 to 3/20/17 2:00 PM to 10:30 AM,3/20/17 11:05 AM,17-00219
9018,U.S. passport left unattended on copy machine ...,Open,Theft,1009 E. 57th St. (Hitchcock RH),2/14/17 9:50 AM,3/20/17 12:26 PM,17-00220
9019,An unauthorized individual sleeping in the bui...,Ex. Cleared,Assault,6019 S. Ingleside (Campus Housing),3/20/17 7:50 AM,3/20/17 3:20 PM,17-00221


In [9]:
# remove nan values
df = df.dropna()

# remove voids (check for error in comments column)
void_err = lambda x: re.search(r'error', x) is None
df = df[df['Comments'].apply(void_err)]
df.shape

(10075, 7)

In [10]:
# lowercase and strip first four columns

for col in df.columns[[0, 1, 2, 3]]:
    df[col] = df[col].apply(lambda x: x.lower().strip())

## Extract GPS coordinate
We use the geopy, a geocoding web service, module to interact with Google Maps API.
We have addresses that need geolocation coordinates.

In [11]:
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut

In [12]:
# get api-key (unique to user, get a key in the link below)
# https://developers.google.com/maps/documentation/geocoding/start
with open('google-maps-api/api-key.txt', 'r') as f:
    key = f.readline()

# setup geolocator using Google Maps API
geolocator = GoogleV3(api_key=key, user_agent="uchicago_surronding_area", format_string="%s, Chicago, IL")

In [13]:
'''
Function takes an address as a string and formats to improve 
likelihood of getting gps coordinates from geocode.

Returns:
    If location found: a tuple containing the latitude & longitude
    If not: an integer code (user defined)
'''
def address2coordinates(address):
    
    # adding "avenue" helps narrow down location
    def addAvenue(address):
        avenues = 'cottage grove,drexel,ingleside,ellis,greenwood,university,woodlawn,kimbark,\
kenwood,dorchester,blackstone,harper,lake park,stony island,cornell,everett'.split(',')
        
        for ave in avenues:
            if re.search(ave, address):
                address += ' avenue'
                break
                
        return address

    
    address = re.sub(r' \(.*\)', '', address.lower())  # remove parenthetical info
    address = re.sub(r' at|and ', ' & ', address)      # fix intersection
    
    try:
        # check if address is between streets
        # e.g. (52nd St. between Greenwood & University)
        pattern = re.findall(r'(\w.*(?= between)) between ((?<=between )\w.*(?= &)) & ((?<=\& )\w.*)', address)[0]
        street_1 = '{} {}'.format(pattern[0], pattern[1])
        street_2 = '{} {}'.format(pattern[0], pattern[2])
        location_1 = geolocator.geocode(addAvenue(street_1), timeout=10)  # server timeout time is 10sec
        location_2 = geolocator.geocode(addAvenue(street_2), timeout=10)
        
        # returns midpoint coordinates between the two locations
        # e.g. (52nd st. greenwood avenue / 52nd st. university avenue)
        return ((location_1.latitude + location_2.latitude)/2, (location_1.longitude + location_2.longitude)/2)
            
    except IndexError:
        # only single address
        location = geolocator.geocode(address, timeout=10)
        if location != None:
            return (location.latitude, location.longitude)
        
    except GeocoderTimedOut:
        # server timeout value if any (check later)
        print('Server timeout!')
    
    except:
        # error value will point to unforseen error if any (check later)
        print('An error occurred!')
        return 999
    
    return 0

In [14]:
# add a progress bar for the next cell (over 10K lines)
from tqdm._tqdm_notebook import tqdm_notebook

# change pandas apply to progress_apply
# tqdm adds minute overhead (~90ns; see tqdm documentation) 
tqdm_notebook.pandas(desc="Geocoding Progress!")  

In [12]:
# create batch and save files as checkpoints
batch = np.linspace(0, df['Location'].shape[0], 6, dtype=int) # each will have 2015 entries
for i in range(1, 6):
    coordinates = df['Location'][batch[i-1]: batch[i]].progress_apply(address2coordinates)
    coordinates.to_csv(('data/coordinates/coordinates_{}.csv'.format(i)))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

In [16]:
# concatenate csv files and save

# use magic command to find path to files
filenames = !find -L data -name "*.csv"

coordinates = pd.concat([pd.read_csv(file, header=None) for file in filenames])
coordinates = coordinates.set_index(0)  # fix index

In [17]:
# where are errors 0 (server timeout) and 999 (other error) if any
server_timeout = np.argwhere(coordinates.values=='0')[:, 0]
error_999 = np.argwhere(coordinates.values=='999')[:, 0]

# how many issues?
print('Number of timeouts:\t{}\nNumber of 999 errors:\t{}'.format(len(server_timeout), len(error_999)))

Number of timeouts:	7
Number of 999 errors:	0


In [18]:
'''
REMOVE:
'Various Campus Buildings' - invalid address
220 W. - lost propert Public Transit
53rd - lost cellphone
5640 S. - lost propert Public Transit

WHY:
Invalid Address or wide street range
'''

# redo timeouts
redo_addresses = df['Location'].values[server_timeout]
redo_addresses

array(['various campus buildings', '220 w. garfield to 1000 e. 55th st.',
       '53rd & kimbark to 60th & ingleside',
       '900 e. 57th st. (knapp center)', '5250 s. harper ct.',
       '5640 s. ellis to 56th & cornell',
       '5525 s. cottage grove (public way)'], dtype=object)

In [19]:
# get geolocation again
redo_list = [address2coordinates(arr) for arr in redo_addresses]
redo_list

[0,
 0,
 0,
 (41.7916469, -87.6034081),
 (41.8000246, -87.58818049999999),
 0,
 (41.794078, -87.60576200000001)]

In [20]:
# pd.to_csv wrote data as a string
# that's why we are converting here
# easy fix using eval() will follow below
redo_list = np.array(list(map(str, redo_list)))[:, np.newaxis]

coordinates.iloc[server_timeout] = redo_list
coordinates.iloc[server_timeout]

,1
0,
107,0
3467,0
4052,0
4593,"(41.7916469, -87.6034081)"
6449,"(41.8000246, -87.58818049999999)"
9010,0
11063,"(41.794078, -87.60576200000001)"


In [21]:
# change string tuple coordinates to tuple with numberic coordinates
coordinates = coordinates[1].apply(eval)  

# save fix merge
coordinates.to_csv('data/coordinates/merged_coordinates.csv')

In [22]:
# add all coordinates to dataframe
df['Coordinates'] = coordinates.values
df.shape

(10075, 8)

In [23]:
#drop rows with coordinate = 0
df = df[df['Coordinates'] != 0]
df['Coordinates'].shape

(10071,)

In [24]:
# some voids weren't picked up 
# remove them
df = df[df['Disposition']!='void']
df.shape

(10032, 8)

In [25]:
# reset index since all bad data is dropped
df.reset_index(drop=True)
df.head()

,Comments,Disposition,Incident,Location,Occurred,Reported,UCPD_ID,Coordinates
0,woman reports losing a canon power shot digita...,closed,lost property,5810 s. university (quad),6/28/10 2:45 PM,7/1/10 12:42 PM,W0731,"(41.7892236, -87.5980197)"
1,unknown person smashed windshield of a rental ...,open,criminal damage to vehicle,61st & drexel,6/29/10 to 7/1/10 3:00 PM to 1:50 PM,7/1/10 2:01 PM,W0732,"(41.7840731, -87.6043863)"
2,unknown person smashed rear passenger's side w...,open,theft from motor vehicle,60th between ingleside & ellis,6/29/10 to 7/1/10 5:30 PM to 12:30 PM,7/1/10 2:48 PM,W0733,"(41.785901249999995, -87.60199659999999)"
3,bicycle taken from bike rack,open,theft,60th and ellis,6/23/10 to 6/29/10 9:00 AM to 5:00 PM,7/1/10 3:13 PM,W0734,"(41.7859081, -87.6011825)"
4,"bottle slipped from man's hands, broke causing...",closed,chemical spill,924 e. 57th st. (bslc),7/1/10 6:15 PM,7/1/10 6:15 PM,W0735,"(41.7919425, -87.60283989999999)"


## Consolidate number of Incidents
There are a lot of incidents and we're interested in a few times. Let's group those that are similar (e.g theft from motor vehicle vs theft). Though similar under the law they are different, however, the main point is here is to see if there is any apparent trend.

In [26]:
# number of unique incidents
df['Incident'].value_counts()

theft                                                       2336
information                                                  821
traffic violation                                            692
lost property                                                649
found property                                               377
liquor law violation                                         262
injured person                                               257
burglary                                                     252
theft from motor vehicle                                     238
criminal damage to property                                  189
battery                                                      166
medical call                                                 152
assist other agency                                          137
information / armed robbery                                  136
information / burglary                                       133
warrant arrest           

In [27]:
# would like to use groupby() but too many options
# let's make something similar

# incident categories
categories = 'lost,theft,assault,burglary,mental health'.split(',')

# find incidents which are similar and group them together
# in a dict, where key=defined category & value=index (like groupby)
categories_data = {}
for i in categories:
    categories_data[i] = df['Incident'].str.extract(r'({})'.format(i), expand=True).dropna().index
    
categories_data['assault']  # preview

Int64Index([   34,    42,    82,   135,   147,   219,   311,   334,   412,
              467,
            ...
            10715, 10745, 10781, 10906, 10957, 11007, 11008, 11019, 11090,
            11114],
           dtype='int64', length=130)

In [28]:
import itertools

In [44]:
# get index w/ all relevant incidents
crime_index = list(itertools.chain.from_iterable(categories_data.values()))

# rename incidents to match those in categories
rename_incidents = [[cat] * len(categories_data[cat]) for cat in categories_data]
rename_incidents = pd.Series(list(itertools.chain.from_iterable(rename_incidents)), index=crime_index, name='Incident')

# change occurred times to datetime64 types
time = df['Occurred'].str.extract(r'^(\d/\d+/\d+ \d+:\d+)( AM| PM)')
time2 = df['Occurred'].str.extract(r'^(\d+/\d+/\d+ \d+:\d+)(AM|PM)').dropna()
time = time.fillna(value=time2)[0] + time.fillna(value=time2)[1]
time = pd.to_datetime(time, errors='coerce').dt.tz_localize(tz='US/Central').rename('Occurred')

# save dataframe w/ imported info

pd.concat([rename_incidents, df['Comments'].loc[crime_index],
           pd.to_datetime(df['Reported'].loc[crime_index], errors='coerce'),
          df['Coordinates'].loc[crime_index], time.loc[crime_index]], axis=1).to_csv('clean_data.csv')